# Digital Assistant: chat-bots

In this case study we build a chatbot prototype using NLP and
ML to understand the user’s intent and have response provided based on underlying logic.


## Content

* [1. Problem Definition](#1)
* [2. Getting Started - Load Libraries and Dataset](#2)
    * [2.1. Load Libraries](#2.1) 
* [3. Training a default chatbot ](#3)  
* [4. Data Preparation for customized chatbot](#4)
* [5.Model Construction and Training](#5)        
    * [5.1. Model Construction](#5.1)
    * [5.2. Building Custom Logic Adapter](#5.2)
    * [5.3. Training the model](#5.3)       
* [6.Model Testing and Usage](#6)           

<a id='1'></a>
# 1. Problem Definition

The goal of this case study is to build a basic prototype of the conversational chatbot
powered by NLP. The primary purpose of this chatbot would be to retrieve the finan‐
cial ratio for the company user is looking for. Such chatbots designed to quickly
retrieve the summary of a stock or instrument may help the user to make a trading
decision.

<a id='2'></a>
# 2. Getting Started- Loading the data and python packages


<a id='2.1'></a>
## 2.1. Loading the python packages
For this case study we use python package - chatterbot. Chatterbot is a python library to create a simple chatbot with minimal programming required. 
Let us chek is the Chatterbot package is present, if not install it. This package is checked separately as it is not included in requirement.txt of this book repository as the package is not used across any other case study of thie book.

In [73]:
import pkg_resources
# import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
# !pip install --upgrade pip setuptools wheel
if 'chatterbot' not in installedPackages :
    !pip install chatterbot==1.0.2

Let us load the chatterbot package

In [74]:
# Load libraries
from chatterbot import ChatBot
from chatterbot.logic import LogicAdapter
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer

In [75]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

Before we move to the customised chatbot, let us develop a chatbot using the defualt features and logic adapters of chatterbot package. 

<a id='3'></a>
## 3 Training a default chatbot 

Before we move on to build a chatbot for customised function avilable in chatterbot. Chatterbot and many other chatbot packages comes with a data utility module that can be used to train the chatbots.

Following is a simple example to get started with ChatterBot in python with the following components. 
* **preprocessors** 
* **logic_adapters** 
* **corpus training** 
* **list training** 

In [76]:
chatB = ChatBot("Trader",
                preprocessors=['chatterbot.preprocessors.clean_whitespace'],
                logic_adapters=['chatterbot.logic.BestMatch',
                                'chatterbot.logic.MathematicalEvaluation'])

# Corpus Training
trainerCorpus = ChatterBotCorpusTrainer(chatB)

#Train based on English Corpus
trainerCorpus.train(
    "chatterbot.corpus.english"
)
# Train based on english greetings corpus
trainerCorpus.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
trainerCorpus.train("chatterbot.corpus.english.conversations")

trainerConversation = ListTrainer(chatB)
#Traing based on conversations

#List training
trainerConversation.train([
    'Help!',
    'Please go to google.com',
    'What is Bitcoin?',
    'It is a decentralized digital currency'
])

# You can train with a second list of data to add response variations
trainerConversation.train([
    'What is Bitcoin?',
    'Bitcoin is a cryptocurrency.'
])


Training ai.yml: [####################] 100%[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hahnsang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/hahnsang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hahnsang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [###############

In [77]:
def converse(quit="quit"):
    user_input = ""
    while user_input != quit:
        user_input = quit
        try:
            user_input = input(">")
        except EOFError:
            print(user_input)
        if user_input:
            while user_input[-1] in "!.":
                user_input = user_input[:-1]
            print(chatB.get_response(user_input))

In [78]:
converse()

No value for search_text was available on the provided input
How are you doing?
No value for search_text was available on the provided input
That is good to hear
No value for search_text was available on the provided input
78964 plus 5970 = 84934
No value for search_text was available on the provided input
dollar: unit of currency in the united states.
No value for search_text was available on the provided input
It is a decentralized digital currency
No value for search_text was available on the provided input
Please go to google.com
No value for search_text was available on the provided input
Did you hear the one about the mountain goats in the andes? It was "ba a a a a a d".
No value for search_text was available on the provided input
If the implementation is easy to explain, it may be a good idea.


In this example, we see a fairly good chatbot which gives us response according to the input that we have given. The first two responses are due to the training on english greetings and conversation corpus. Additionally the response to "tell me a joke" and "what is a dollar" are due to the training on the english corpus. The computation in the forth line is the result of the chatbot being trained on the Mathematical Evaluation logical adapter. The response to "Help" and "What is a bitcoin" are the result of the customised list trainers. 

Given, that we have already have a customised chatbot, we move on to create a chatbot which is designed to give us the financial ratios of a company based on a customised logical adapter.

<a id='4'></a>
# 4. Data Preparation for customized chatbot

The purpose of performing the data preparation is to use it for training through logic adapter.The details are under https://chatterbot.readthedocs.io/en/stable/logic/create-a-logic-adapter.html. Given the logic adapter need to be in a separate file from the chat bot, we perform the step of data preparation in the module **financial_ratio_adapter.py** where logic adapter is created.


<a id='5'></a>
# 5. Model construction and training

<a id='5.1'></a>
## 5.1 and 5.2 Model optimization function and building custom logic adapter
Step 4.2 and 4.2 are shown in the module **financial_ratio_adapter.py**, given the logic adapter need to be in a separate file from the chat bot. In the next step we train the chatbot, which trains it on the customised logic adapter. 

<a id='5.3'></a>
## 5.3. Training the model

In this step we combine all the components (i.e. preprocessor, custom logical adapter, list and corpus trainer) with the custom logical adapter (financial_ratio_adapter.FinancialRatioAdapter) that we have created. 

In [79]:
#Here we add 
chatbot = ChatBot(
    "My ChatterBot",
    preprocessors=['chatterbot.preprocessors.clean_whitespace'],
    logic_adapters=[
        'financial_ratio_adapter.FinancialRatioAdapter',
        'chatterbot.logic.MathematicalEvaluation',
        'chatterbot.logic.BestMatch'
    ]
)

#Train based on English Corpus
trainerCorpus.train(
    "chatterbot.corpus.english"
)
# Train based on english greetings corpus
trainerCorpus.train("chatterbot.corpus.english.greetings")

# Train based on the english conversations corpus
trainerCorpus.train("chatterbot.corpus.english.conversations")

trainerConversation = ListTrainer(chatB)
#Traing based on conversations

trainerConversation.train([
    'Help!',
    'Please go to google.com',
    'What is Bitcoin?',
    'It is a decentralized digital currency'
])

# You can train with a second list of data to add response variations
trainerConversation.train([
    'What is Bitcoin?',
    'Bitcoin is a cryptocurrency.'
])


Training ai.yml: [####################] 100%
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hahnsang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/hahnsang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hahnsang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [###############

As we can see the training was not only for the FinancialRatioAdapter, but also for the list and corpus trainer. Let us move to the model testing.

<a id='6'></a>
# 6. Model Testing and Usage

In [80]:
def converse(quit="quit"):
    user_input = ""
    while user_input != quit:
        user_input = quit
        try:
            user_input = input(">")
        except EOFError:
            print(user_input)
        if user_input:
            while user_input[-1] in "!.":
                user_input = user_input[:-1]
            print(chatbot.get_response(user_input))

In [81]:
converse()

No value for search_text was available on the provided input
Sorry! Could not figure out what the user wants
No value for search_text was available on the provided input
https://www.zacks.com/stock/chart/DAL/fundamental/pe-ratio-ttm
					  
No value for search_text was available on the provided input
https://www.zacks.com/stock/chart/C/fundamental/return-on-equity-ttm
					  
No value for search_text was available on the provided input
It is a decentralized digital currency
No value for search_text was available on the provided input
786940 plus 75868 = 862808
No value for search_text was available on the provided input
Much the same as being a human, except that we lack all emotions, dreams, aspirations, creativity, ambition, and above all subjectivity.
No value for search_text was available on the provided input
If the implementation is easy to explain, it may be a good idea.


The custom logic adaptor for our Chatter bot, finds a RATIO or a COMPANY in the sentence using our NLP model. If the model finds exactly one COMPANY and exactly one RATIO, it con structs a url to guide the user. Additionally other logical adpater such as mathematical evaluation, and curpus and list trainer work as expected as well. 

**Conclusion**

In this case study, we have learned how to make a chatbot in python using the ChatterBot library. We learnt how to build a custom NLP based model focusing on NER(Named Entity Recognition) and use in a chatbot.


In order to train a blank model, one must have a substantial training dataset. In this
case study, we looked at patterns available to us and used them to generate training
samples. 

This case study is a demo project, and significant enhancements can be made for each
component to extend it for a wide variety of tasks. Additional preprocessing steps can
be added to have cleaner data to work with. 

Overall, this case study provides an introduction to all the aspects of chatbot development. Although, it is a very simple bot, it’s a good starting point to use NLP to create
chatbots.


